In [2]:
import os
import cv2
import random
import shutil
import torchvision.transforms as transforms


from xeception_2x1ch import *
from xeception import *
from googlenetv4 import *
import pandas as pd
import numpy as np

d:\SHkim\contactlensCC_wxpython\wx_venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
radius_list = []
def concat_origin_img(white_img, black_img):
    white_img = 255 - white_img

    img_sum = np.zeros((white_img.shape[0],white_img.shape[1],3), dtype=np.uint8)
    img_sum[:,:,0] = white_img
    img_sum[:,:,1] = black_img
    
    return img_sum

def find_circles(img_origin):
    global radius_list
    try:
        img = cv2.cvtColor(img_origin, cv2.COLOR_BGR2GRAY)
        #가우시안필터
        gaus = cv2.GaussianBlur(img, (5, 5), 7)

        #적응형이진화
        thres = cv2.adaptiveThreshold(gaus, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 9, 5)
        thres = cv2.GaussianBlur(thres, (5, 5), 7)
        
        circles = cv2.HoughCircles(thres, cv2.HOUGH_GRADIENT, 1, 1200, param1 = 250, param2 = 60, minRadius = 720, maxRadius = 800)
    except Exception as e:
        print(e)
        return False, img    
    try:
        if len(circles) > 0:
            for i in circles[0]:
                cx = i[0]
                cy = i[1]
                radius = i[2]
                radius_list.append(radius)
    except:
        cx = img_origin.shape[1]//2
        cy = img_origin.shape[0]//2
        print("cannot find circles")

    #이미지 ROI 추출
    margin = 40 # 여백
    s_radius = 740
    x_min = int(cx - s_radius - margin)
    y_min = int(cy - s_radius - margin)
    x_max = int(cx + s_radius + margin)
    y_max = int(cy + s_radius + margin)
    img_roi = img_origin[y_min:y_max,x_min:x_max]
    img_roi = cv2.resize(img_roi,(1500,1500))
    # show_subplot_img([img_roi], 15)
        
    return True, img_roi

In [3]:
testimg_base_dir = 'E:/test_temp/tint/'
target_dir = '/test'
dir_list = os.listdir(testimg_base_dir)
df = pd.read_csv('dataset.csv')
print(dir_list)
for dirs in dir_list:
    os.makedirs(testimg_base_dir+dirs+target_dir, exist_ok=True)
    img_list = [x for x in os.listdir(testimg_base_dir+dirs) if x.endswith(('jpg', 'bmp'))]
    for files in img_list:
        file_key = '_'.join(files.split("_")[:-1])
        contains_string = df['id'].str.contains(file_key).any()
        if contains_string:
            continue
        else:
            try:
                w_name = file_key+'_0.jpg'
                b_name = file_key+'_1.jpg'
                w_img = cv2.imread(testimg_base_dir+dirs +'/'+ w_name , 0)
                b_img = cv2.imread(testimg_base_dir+dirs +'/'+ b_name , 0)
                img_sum = concat_origin_img(w_img, b_img)

                ref, img_roi = find_circles(img_sum)
                if ref:
                    cv2.imwrite(testimg_base_dir+dirs+target_dir + '/' + b_name, img_roi)
                else:
                    continue
            except:
                continue

['tint_b-bubble', 'tint_b-edge', 'tint_Broken', 'tint_burr']
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles
cannot find circles

In [28]:
sum(radius_list) / len(radius_list)

741.4494524826812

In [6]:
import test
import numpy as np
import time
import shutil

def copy_to_savefolder(img_path_list, result_list, class_list, save_dir):
    for n,result in enumerate(result_list):
        for i,r in enumerate(result):
            if r:
                shutil.copy(img_path_list[n], save_dir + '/' + class_list[i]+ '/' + os.path.basename(img_path_list[n]))

class_list = ['good', 'b_edge', 'burr', 'borken', 'b_bubble', 'etc', 'no_lens']

base_path = r'E:\test_temp\for_testing'
save_dir = r'E:\test_temp\test_result\xec'

for clsss in class_list:
    os.makedirs(save_dir + '/' + clsss, exist_ok=True)

for dirs in os.listdir(base_path):
    print(dirs)
    
    start_time = time.time()
    img_path = base_path + '/' + dirs
    file_list, out = test.classfication(img_path=img_path)
    copy_to_savefolder(file_list, out, class_list=class_list, save_dir=save_dir)
    results_array = np.array(out)

    # count number of True values per column
    column_counts = np.sum(results_array, axis=0)

    # count number of True values in total
    total_count = len(out)

    print("Count per column:", column_counts)
    print("Total count:", total_count)
    print("total time : ", time.time() - start_time)
    print("average time : ", (time.time() - start_time) / total_count)

broken
E:\test_temp\for_testing/broken is a directory.
E:\test_temp\for_testing/broken is a directory.
Count per column: [ 19  44   0 333   5   2  10]
Total count: 560
total time :  37.52670168876648
average time :  0.06701196730136871
burr
E:\test_temp\for_testing/burr is a directory.
E:\test_temp\for_testing/burr is a directory.
Count per column: [  1 196 737  95   9  35  16]
Total count: 952
total time :  63.59628176689148
average time :  0.06680281698202886
b_bubble
E:\test_temp\for_testing/b_bubble is a directory.
E:\test_temp\for_testing/b_bubble is a directory.
Count per column: [  34   13    0   11 1083   62   20]
Total count: 1259
total time :  82.53267645835876
average time :  0.06555415127748909
b_edge
E:\test_temp\for_testing/b_edge is a directory.
E:\test_temp\for_testing/b_edge is a directory.
Count per column: [168 368   0   3   0   5   4]
Total count: 578
total time :  39.489999532699585
average time :  0.06832179849948025


In [5]:
import test
import numpy as np
import time
import random
import cv2

base_path = r'E:\test_temp\for_testing'

for dirs in os.listdir(base_path):
    print(dirs)
    img_path = base_path + '/' + dirs
    
    rnum1, rnum2 = random.sample(range(len(os.listdir(img_path))), 2)
    print(rnum1, rnum2)

    img1 = cv2.imread(base_path + '/' + dirs + '/' + os.listdir(img_path)[rnum1])
    img2 = cv2.imread(base_path + '/' + dirs + '/' + os.listdir(img_path)[rnum2])

    img_l =[img1, img2]
    
    file_list, out = test.classfication(img_path=[img1],selected_model='googlenetv4', weight_path="E:/weights/classification_weight/train_13/60_weight.pt")
    print(out)
    break

broken
78 548
List of OpenCV images received!
[[False, False, False, True, False, False, False]]


In [8]:
import test
import numpy as np
import time

base_path = r'E:\test_temp\for_testing'
save_dir = r'E:\test_temp\test_result\ggn'

for clsss in class_list:
    os.makedirs(save_dir + '/' + clsss, exist_ok=True)

for dirs in os.listdir(base_path):
    print(dirs)
    start_time = time.time()
    img_path = base_path + '/' + dirs
    file_list, out = test.classfication(img_path=img_path,selected_model='googlenetv4', weight_path="E:/weights/classification_weight/train_13/60_weight.pt")
    copy_to_savefolder(file_list, out, class_list=class_list, save_dir=save_dir)
    results_array = np.array(out)

    # count number of True values per column
    column_counts = np.sum(results_array, axis=0)

    # count number of True values in total
    total_count = len(out)

    print("Count per column:", column_counts)
    print("Total count:", total_count)
    print("total time : ", time.time() - start_time)
    print("average time : ", (time.time() - start_time) / total_count)

broken
E:\test_temp\for_testing/broken is a directory.
E:\test_temp\for_testing/broken is a directory.
Count per column: [  4  23   0 525   4   2  11]
Total count: 560
total time :  26.867186546325684
average time :  0.04797711883272444
burr
E:\test_temp\for_testing/burr is a directory.
E:\test_temp\for_testing/burr is a directory.
Count per column: [  3  92 880  44   1  16  18]
Total count: 952
total time :  43.57608127593994
average time :  0.04577319461758397
b_bubble
E:\test_temp\for_testing/b_bubble is a directory.
E:\test_temp\for_testing/b_bubble is a directory.
Count per column: [  34   14   10  133 1096   25   25]
Total count: 1259
total time :  55.822723150253296
average time :  0.04433893816541167
b_edge
E:\test_temp\for_testing/b_edge is a directory.
E:\test_temp\for_testing/b_edge is a directory.
Count per column: [ 50 498   0   8   0   9   5]
Total count: 578
total time :  28.8340322971344
average time :  0.04988586902618408
